In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [6]:
features_list = ['Num_Enq_all_loans_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_36M',
'num_CREDIT CARD_enq_last_18M',
'Total_outstanding_Balance_CREDIT CARD',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'num_PL_LE_1_Lac_with_no_delinquency',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'Utilization_all',
'num_of_loans_le_1lac_non_credit',
                 
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'num_debits_gt_10000_M3',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',
'cash_withdrawal_sum',
'num_cash_withdrawal_last_1_months',
'bounced_transactions_count',
'min_balance_last_3_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'avg_num_credits_last_2_months',
'Max_Credit_Avg_M1_M2_M3',
'ratio_3th_day_balance_to_max_credit_M2',
'ratio_active_loan_vs_salary',
'num_self_trans_last_3_months',

'deviation_from_mean_income#last_30_Days',
'age',



'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
                 

'dm206s',
'g406s',
'balmag01',
'aggs907',

                 
'month_year',
'G15_B24']

In [7]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3040112/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3040112/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 46), (13738, 46), (20606,), (13738,))

In [10]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 46) (4122, 46) (13738, 46)


In [11]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [12]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 5,
    'alpha': 4.5,
    'gamma': 4.8,
    'random_state': 42
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 300,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.57844	eval-auc:0.54666
[10]	train-auc:0.61992	eval-auc:0.60073
[20]	train-auc:0.62871	eval-auc:0.61056
[30]	train-auc:0.64605	eval-auc:0.62707
[40]	train-auc:0.64745	eval-auc:0.62617
[50]	train-auc:0.64811	eval-auc:0.63292
[60]	train-auc:0.64989	eval-auc:0.63269
[70]	train-auc:0.65186	eval-auc:0.63550
[80]	train-auc:0.65519	eval-auc:0.63591
[90]	train-auc:0.65654	eval-auc:0.63977
[100]	train-auc:0.65725	eval-auc:0.64337
[110]	train-auc:0.66102	eval-auc:0.64499
[120]	train-auc:0.66088	eval-auc:0.64545
[130]	train-auc:0.66216	eval-auc:0.64604
[140]	train-auc:0.66415	eval-auc:0.64844
[150]	train-auc:0.66494	eval-auc:0.65063
[160]	train-auc:0.66546	eval-auc:0.65209
[170]	train-auc:0.66596	eval-auc:0.65265
[180]	train-auc:0.66702	eval-auc:0.65393
[190]	train-auc:0.66792	eval-auc:0.65409
[200]	train-auc:0.66907	eval-auc:0.65461
[210]	train-auc:0.67012	eval-auc:0.65611
[220]	train-auc:0.67059	eval-auc:0.65622
[230]	train-auc:0.67116	eval-auc:0.65757
[240]	train-auc:0.67172	eva

In [13]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.68, Gini: 0.35, KS: 0.25
Eval AUC: 0.66, Gini: 0.32, KS: 0.25
Test AUC: 0.65, Gini: 0.29, KS: 0.23
OOT AUC: 0.65, Gini: 0.30, KS: 0.22


In [14]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.175258         0.128707        0.224093        0.105578   1649   
9          0.083738         0.095048        0.105576        0.086019   1648   
8          0.087379         0.079361        0.086013        0.073607   1648   
7          0.063069         0.069511        0.073599        0.066252   1649   
6          0.061286         0.064037        0.066252        0.062054   1648   
5          0.047937         0.060329        0.062052        0.058413   1648   
4          0.052759         0.055615        0.058412        0.053099   1649   
3          0.034587         0.051432        0.053096        0.049933   1648   
2          0.034587         0.048489        0.049931        0.046886   1648   
1          0.013948         0.044907        0.046885        0.041256   1649   

        label_good  label_bad  
Decile                         
10            1360        289  
9             1510        138  
8             1504        144  
7             1545        104  
6             1547        101  
5             1569         79  
4             1562         87  
3             1591         57  
2             1591         57  
1             1626         23

In [15]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.131004         0.126900        0.224809        0.105501   1374   
9          0.082969         0.094684        0.105476        0.085634   1374   
8          0.077875         0.079277        0.085633        0.073558   1374   
7          0.078660         0.069386        0.073555        0.065986   1373   
6          0.061863         0.063883        0.065980        0.061975   1374   
5          0.049491         0.060055        0.061974        0.057916   1374   
4          0.036417         0.055128        0.057904        0.052810   1373   
3          0.036390         0.051199        0.052807        0.049732   1374   
2          0.033479         0.048310        0.049727        0.046756   1374   
1          0.024745         0.044831        0.046755        0.041256   1374   

        label_good  label_bad  
Decile                         
10            1194        180  
9             1260        114  
8             1267        107  
7             1265        108  
6             1289         85  
5             1306         68  
4             1323         50  
3             1324         50  
2             1328         46  
1             1340         34

In [16]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.160962         0.128537        0.225996        0.107005   1081   
9          0.103704         0.096892        0.107000        0.087292   1080   
8          0.102778         0.080156        0.087290        0.074442   1080   
7          0.087963         0.070067        0.074436        0.066416   1080   
6          0.076852         0.064342        0.066411        0.062489   1080   
5          0.055556         0.060839        0.062487        0.059305   1080   
4          0.068519         0.056711        0.059303        0.054114   1080   
3          0.044444         0.052342        0.054113        0.050790   1080   
2          0.032407         0.049301        0.050789        0.047674   1080   
1          0.027752         0.045407        0.047674        0.041256   1081   

        label_good  label_bad  
Decile                         
10             907        174  
9              968        112  
8              969        111  
7              985         95  
6              997         83  
5             1020         60  
4             1006         74  
3             1032         48  
2             1045         35  
1             1051         30

In [17]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
38                                              g406s       121.0
34                                         days_on_pg       115.0
22                         bounced_transactions_count        69.0
0                           Num_Enq_all_loans_last_9M        60.0
17                          Num_Enq_non_BANK_last_12M        57.0
33  MEAN(raw.amount WHERE device_os_category = Win...        43.0
1                  Num_of_PERSONAL LOAN_opened_last9M        36.0
12                                    Utilization_all        31.0
20                                    upi_trans_count        30.0
3                      num_CONSUMER LOAN_enq_last_36M        29.0
7                          Age_of_oldest_open_account        25.0
36                              amt_first_android_txn        22.0
31                                                age        21.0
26                      avg_num_credits_last_2_months        14.0
24        

In [18]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILA_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_BBILA_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_BBILA_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_BBILA_Oot_Pred_1524.csv',index=False)


In [19]:
#Save the Model
xgb_model.save_model('LeoPayu_BBILAP_15_in_24mob.json')
print('Model Saved')

Model Saved
